<a href="https://colab.research.google.com/github/DavidGerardo2/Actividades/blob/main/AF_3_PIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AF3: PIA
# Datos del equipo:
# - David Gerardo Guipzot Sanchez - 2173741 - Martes N4,N5,N6
# - Diego Armando Bustos Montantes - 2063883 - Martes N4,N5,N6
# Fecha: 21 de noviembre del 2025

In [ ]:
# Codigo de los servos

#include <Servo.h>

Servo servo1;
Servo servo2;

String comando = "";
String ultimo_comando = "";

unsigned long tiempo_inicio = 0;
bool ejecutando = false;

const int s1_ini = 170;
const int s2_ini = 10;

void setup() {
  Serial.begin(115200);

  servo1.attach(3);
  servo2.attach(4);

  servo1.write(s1_ini);
  servo2.write(s2_ini);

  Serial.println("Listo.");
}

void loop() {

  // LECTURA
  if (Serial.available() > 0 && !ejecutando) {

    comando = Serial.readStringUntil('\n');
    comando.trim();

    if (comando.length() == 1) {

      // permitir repetir el mismo comando SOLO si ya terminó el ciclo
      if (comando == ultimo_comando && ejecutando == false) {
        // permitido
      }
      else if (comando == ultimo_comando && ejecutando == true) {
        // aún está en curso → ignorar
        return;
      }

      ultimo_comando = comando;
      ejecutando = true;
      tiempo_inicio = millis();

      Serial.print("Ejecutando: ");
      Serial.println(comando);

      if (comando == "R") {
        servo1.write(40);
        servo2.write(50);
      }
      else if (comando == "A") {
        servo1.write(135);
        servo2.write(160);
      }
      else if (comando == "C" || comando == "B") {
        servo1.write(100);
        servo2.write(100);
      }
    }
  }

  // TIEMPO → ahora 4000 ms (4 segundos)
  if (ejecutando && millis() - tiempo_inicio >= 4000) {
    servo1.write(s1_ini);
    servo2.write(s2_ini);

    ejecutando = false;

    Serial.println("Servos regresaron.");
  }
}


In [ ]:
# Codigo de TRUEESP32

/* Includes ---------------------------------------------------------------- */
#include <klot_inferencing.h>
#include "edge-impulse-sdk/dsp/image/image.hpp"
#include "esp_camera.h"

// Select camera model
#define CAMERA_MODEL_AI_THINKER // Has PSRAM

#if defined(CAMERA_MODEL_AI_THINKER)
#define PWDN_GPIO_NUM     32
#define RESET_GPIO_NUM    -1
#define XCLK_GPIO_NUM      0
#define SIOD_GPIO_NUM     26
#define SIOC_GPIO_NUM     27

#define Y9_GPIO_NUM       35
#define Y8_GPIO_NUM       34
#define Y7_GPIO_NUM       39
#define Y6_GPIO_NUM       36
#define Y5_GPIO_NUM       21
#define Y4_GPIO_NUM       19
#define Y3_GPIO_NUM       18
#define Y2_GPIO_NUM        5
#define VSYNC_GPIO_NUM    25
#define HREF_GPIO_NUM     23
#define PCLK_GPIO_NUM     22

#else
#error "Camera model not selected"
#endif

#define EI_CAMERA_RAW_FRAME_BUFFER_COLS           320
#define EI_CAMERA_RAW_FRAME_BUFFER_ROWS           240
#define EI_CAMERA_FRAME_BYTE_SIZE                 3

static bool debug_nn = false;
static bool is_initialised = false;
uint8_t *snapshot_buf;

static camera_config_t camera_config = {
    .pin_pwdn = PWDN_GPIO_NUM,
    .pin_reset = RESET_GPIO_NUM,
    .pin_xclk = XCLK_GPIO_NUM,
    .pin_sscb_sda = SIOD_GPIO_NUM,
    .pin_sscb_scl = SIOC_GPIO_NUM,

    .pin_d7 = Y9_GPIO_NUM,
    .pin_d6 = Y8_GPIO_NUM,
    .pin_d5 = Y7_GPIO_NUM,
    .pin_d4 = Y6_GPIO_NUM,
    .pin_d3 = Y5_GPIO_NUM,
    .pin_d2 = Y4_GPIO_NUM,
    .pin_d1 = Y3_GPIO_NUM,
    .pin_d0 = Y2_GPIO_NUM,
    .pin_vsync = VSYNC_GPIO_NUM,
    .pin_href = HREF_GPIO_NUM,
    .pin_pclk = PCLK_GPIO_NUM,

    .xclk_freq_hz = 20000000,
    .ledc_timer = LEDC_TIMER_0,
    .ledc_channel = LEDC_CHANNEL_0,

    .pixel_format = PIXFORMAT_JPEG,
    .frame_size = FRAMESIZE_QVGA,

    .jpeg_quality = 12,
    .fb_count = 1,
    .fb_location = CAMERA_FB_IN_PSRAM,
    .grab_mode = CAMERA_GRAB_WHEN_EMPTY,
};

/* Function definitions ------------------------------------------------------- */
bool ei_camera_init(void);
void ei_camera_deinit(void);
bool ei_camera_capture(uint32_t img_width, uint32_t img_height, uint8_t *out_buf);

/**
* @brief      Arduino setup function
*/
void setup()
{
    Serial.begin(115200);
    while (!Serial);

    Serial.println("Edge Impulse Inferencing Demo");

    if (ei_camera_init() == false) {
        ei_printf("Failed to initialize Camera!\r\n");
    }
    else {
        ei_printf("Camera initialized\r\n");
    }

    ei_printf("\nStarting continious inference in 2 seconds...\n");
    ei_sleep(2000);
}

/* -------------------------
   NUEVAS VARIABLES GLOBALES
   ------------------------- */
unsigned long lastSend = 0;         // temporizador del envío de 4.5s
float mejor_en_ciclo = 0.0f;       // mejor puntuación acumulada en el ciclo
char mejor_letra = 'N';            // letra candidata con mejor puntuación en el ciclo
const unsigned long INTERVALO_MS = 4500; // 4.5 segundos
/* ------------------------- */

/**
* @brief      Get data and run inferencing
*/
void loop()
{
    // Mantengo la misma llamada a ei_sleep que usa tu proyecto Edge Impulse
    if (ei_sleep(5) != EI_IMPULSE_OK) {
        return;
    }

    // Reserva buffer para imagen original (usa tus macros)
    snapshot_buf = (uint8_t*)malloc(EI_CAMERA_RAW_FRAME_BUFFER_COLS * EI_CAMERA_RAW_FRAME_BUFFER_ROWS * EI_CAMERA_FRAME_BYTE_SIZE);

    if(snapshot_buf == nullptr) {
        ei_printf("ERR: Failed to allocate snapshot buffer!\n");
        return;
    }

    // Preparo la señal para el clasificador
    ei::signal_t signal;
    signal.total_length = EI_CLASSIFIER_INPUT_WIDTH * EI_CLASSIFIER_INPUT_HEIGHT;
    signal.get_data = &ei_camera_get_data;

    // Captura la imagen y convierte si hace falta dentro de ei_camera_capture
    if (ei_camera_capture((size_t)EI_CLASSIFIER_INPUT_WIDTH, (size_t)EI_CLASSIFIER_INPUT_HEIGHT, snapshot_buf) == false) {
        ei_printf("Failed to capture image\r\n");
        free(snapshot_buf);
        return;
    }

    ei_impulse_result_t result = { 0 };

    // Ejecuta el clasificador
    EI_IMPULSE_ERROR err = run_classifier(&signal, &result, debug_nn);
    if (err != EI_IMPULSE_OK) {
        ei_printf("ERR: Failed to run classifier (%d)\n", err);
        free(snapshot_buf);
        return;
    }

    // --- PROCESAMIENTO: actualizar mejor candidato DEL CICLO ---
    // Para object detection: buscamos la bounding box con mayor confidence
#if EI_CLASSIFIER_OBJECT_DETECTION == 1

    for (uint32_t i = 0; i < result.bounding_boxes_count; i++) {
        ei_impulse_result_bounding_box_t bb = result.bounding_boxes[i];
        if (bb.value > mejor_en_ciclo) {
            mejor_en_ciclo = bb.value;
            // guardamos sólo la primera letra de la etiqueta
            if (bb.label != nullptr && bb.label[0] != '\0') {
                mejor_letra = bb.label[0];
            }
        }
    }

#else
    // Para clasificación simple: buscamos la clase con mayor probabilidad
    for (uint16_t i = 0; i < EI_CLASSIFIER_LABEL_COUNT; i++) {
        float v = result.classification[i].value;
        if (v > mejor_en_ciclo) {
            mejor_en_ciclo = v;
            // guardamos la primera letra del nombre de la clase
            mejor_letra = ei_classifier_inferencing_categories[i][0];
        }
    }
#endif

    // liberamos el buffer de snapshot (tu código ya lo hacía al final)
    free(snapshot_buf);

    // --- ENVIO: sólo UNA letra cada INTERVALO_MS ---
    if ((millis() - lastSend) >= INTERVALO_MS) {
        char a_enviar = 'N';

        // Si en el ciclo se detectó algo con puntuación > 0, enviamos la mejor letra
        if (mejor_en_ciclo > 0.0f) {
            a_enviar = mejor_letra;
        } else {
            a_enviar = 'N';
        }

        // Enviar exactamente UNA LÍNEA (una letra + newline)
        Serial.println(a_enviar);

        // Reset para el siguiente ciclo
        mejor_en_ciclo = 0.0f;
        mejor_letra = 'N';
        lastSend = millis();
    }

    // fin del loop
}

/**
 * @brief   Setup image sensor & start streaming
 */
bool ei_camera_init(void) {

    if (is_initialised) return true;

    esp_err_t err = esp_camera_init(&camera_config);
    if (err != ESP_OK) {
      Serial.printf("Camera init failed with error 0x%x\n", err);
      return false;
    }

    sensor_t * s = esp_camera_sensor_get();
    if (s->id.PID == OV3660_PID) {
      s->set_vflip(s, 1);
      s->set_brightness(s, 1);
      s->set_saturation(s, 0);
    }

    is_initialised = true;
    return true;
}

void ei_camera_deinit(void) {
    esp_err_t err = esp_camera_deinit();
    if (err != ESP_OK) {
        ei_printf("Camera deinit failed\n");
        return;
    }
    is_initialised = false;
}

bool ei_camera_capture(uint32_t img_width, uint32_t img_height, uint8_t *out_buf) {

    if (!is_initialised) {
        ei_printf("ERR: Camera is not initialized\r\n");
        return false;
    }

    camera_fb_t *fb = esp_camera_fb_get();

    if (!fb) {
        ei_printf("Camera capture failed\n");
        return false;
    }

    bool converted = fmt2rgb888(fb->buf, fb->len, PIXFORMAT_JPEG, snapshot_buf);

    esp_camera_fb_return(fb);

    if(!converted){
       ei_printf("Conversion failed\n");
       return false;
    }

    if ((img_width != EI_CAMERA_RAW_FRAME_BUFFER_COLS)
        || (img_height != EI_CAMERA_RAW_FRAME_BUFFER_ROWS)) {

        ei::image::processing::crop_and_interpolate_rgb888(
            out_buf,
            EI_CAMERA_RAW_FRAME_BUFFER_COLS,
            EI_CAMERA_RAW_FRAME_BUFFER_ROWS,
            out_buf,
            img_width,
            img_height);
    }

    return true;
}

static int ei_camera_get_data(size_t offset, size_t length, float *out_ptr)
{
    size_t pixel_ix = offset * 3;
    size_t pixels_left = length;
    size_t out_ptr_ix = 0;

    while (pixels_left != 0) {
        out_ptr[out_ptr_ix] =
            (snapshot_buf[pixel_ix + 2] << 16) +
            (snapshot_buf[pixel_ix + 1] << 8) +
            snapshot_buf[pixel_ix];

        out_ptr_ix++;
        pixel_ix+=3;
        pixels_left--;
    }
    return 0;
}

#if !defined(EI_CLASSIFIER_SENSOR) || EI_CLASSIFIER_SENSOR != EI_CLASSIFIER_SENSOR_CAMERA
#error "Invalid model for current sensor"
#endif


In [ ]:
# Codigo para la activacion del prototipo

import serial
import time

try:
    esp32 = serial.Serial("COM4", 115200, timeout=0)
    arduino = serial.Serial("COM5", 115200, timeout=0)

    esp32.setDTR(False)
    esp32.setRTS(False)

except Exception as e:
    print("\n⚠️ ERROR AL ABRIR PUERTOS:")
    print(e)
    input("\nPresiona ENTER para salir...")
    exit()

print("Puente activo: reenviando SOLO letras válidas (A, B, C, R).\n")

buffer = b""
letras_validas = ["A", "B", "C", "R"]

while True:
    try:
        while esp32.in_waiting:
            c = esp32.read()
            if not c:
                continue

            # Salto de línea = procesar
            if c in b"\r\n":
                if buffer:
                    linea = buffer.decode(errors='ignore').strip()
                    buffer = b""

                    # Si la línea es SOLO 1 letra válida → reenvía
                    if len(linea) == 1 and linea in letras_validas:
                        print("ESP32 →", linea)
                        arduino.write((linea + "\n").encode())

                continue

            buffer += c

        time.sleep(0.01)

    except Exception as e:
        print("\n⚠️ ERROR DURANTE EJECUCIÓN:")
        print(e)
        input("\nPresiona ENTER para salir...")
        break


